In [1]:
import torch.optim as optim
import torch.nn.functional as F

import bayescache.api as api
from bayescache.data import P3B3
from bayescache.models import mtcnn
from bayescache.api.source import TrainingData
from bayescache.callbacks.time_tracker import TimeTracker

In [2]:
hparams = mtcnn.Hyperparameters()
# Update hyperparameters for the Synthetic data.
hparams.vocab_size = 4014
hparams.max_sent_len = 1500

model = mtcnn.new(hparams)

optimizer = optim.RMSprop(model.parameters(), lr=7.0e-4, eps=1e-3)

In [11]:
learner = api.Learner(device='cpu', model=model)
#learner.summary()

In [13]:
train = P3B3(root='/home/ygx/data', partition='train', download=True)
test = P3B3(root='/home/ygx/data', partition='test', download=True)

source = TrainingData(train_source=train, val_source=test, num_workers=2, batch_size=4)
source.train_source

Dataset P3B3
    Number of datapoints: 8000
    Split: train
    Root Location: /home/ygx/data

In [14]:
metrics = learner.metrics()
callbacks = [TimeTracker()]

training_info = api.TrainingInfo(
    start_epoch_idx=0,
    run_name='test',
    metrics=metrics,
    callbacks=callbacks
)

In [20]:
training_info.on_train_begin()


In [21]:
for global_epoch_idx in range(training_info.start_epoch_idx + 1, 1 + 1):
    epoch_info = api.EpochInfo(
        training_info=training_info,
        global_epoch_idx=global_epoch_idx,
        batches_per_epoch=source.train_iterations_per_epoch(),
        optimizer=optimizer
    )

    # Execute learning
    learner.train_epoch(epoch_info, source)


Training:   0%|          | 0/2000 [00:00<?, ?iter/s]

RuntimeError: index out of range at /pytorch/aten/src/TH/generic/THTensorEvenMoreMath.cpp:191

### Diagnosing Loss

In [25]:
import torch
from torch.utils.data import DataLoader

In [26]:
trainloader = DataLoader(train)

In [41]:
for idx, (data, label) in enumerate(trainloader):
    logits = model(data)
    losses = []
    print(label)
    for i in range(len(label[0])):
        print(i)
        loss = F.cross_entropy(logits[i], torch.tensor((label[:,i],)))
        losses.append(loss)
    #loss = sum(loss)
    if idx == 0:
        break 

tensor([[5, 0, 2, 2]])
0
1
2


RuntimeError: Assertion `cur_target >= 0 && cur_target < n_classes' failed.  at /pytorch/aten/src/THNN/generic/ClassNLLCriterion.c:93

In [28]:
logits

[tensor([[ 3.1007,  2.4793, -1.5332,  1.0929,  0.5571, -1.5802]],
        grad_fn=<AddmmBackward>),
 tensor([[-0.0625,  2.6144]], grad_fn=<AddmmBackward>),
 tensor([[1.3258, 3.2886]], grad_fn=<AddmmBackward>),
 tensor([[-2.9914,  0.0363, -0.4757]], grad_fn=<AddmmBackward>)]

In [29]:
torch.tensor((label[0][0],))
label[:,2]

tensor([2])

In [30]:
F.cross_entropy(logits[1], torch.tensor((label[:,1],)))

tensor(2.7434, grad_fn=<NllLossBackward>)

In [ ]:
logits

In [ ]:
logits[0].shape
x = label[0][0].item()
target = torch.tensor((x,))
target

In [ ]:
input = torch.randn(1, 6, requires_grad=True)
target = torch.randint(6, (1,), dtype=torch.int64)

In [ ]:
input.shape
target

### Shape of Things to Come:

In [ ]:
data, labels = train.load_data()

In [ ]:
len(labels)

In [ ]:
len(labels[:,0])

In [ ]:
import numpy as np
unq, counts = np.unique(labels[:,0], return_counts=True)

In [ ]:
unq

In [ ]:
counts

In [ ]:
data[0].shape

In [ ]:
sum(counts)

In [ ]:
data.shape

In [ ]:
labels

In [ ]:
data